In [2]:
import numpy as np
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

# transpose 연습

In [14]:
ct = np.arange(7*5*2).reshape(7,5,2)

print(np.transpose(ct).shape)
print(np.transpose(ct, [1,2,0]).shape)

# m*n -> transpose => n*m
# i*j*k => transpose => 다양한 형태로 가능

(2, 5, 7)
(5, 2, 7)


In [26]:
t = [i for i in range(1,25)]
t = np.reshape(t,[2,3,4])
t = t[-1]

print(t)

[[13 14 15 16]
 [17 18 19 20]
 [21 22 23 24]]


In [ ]:
# case study 1 : 4글자로 구성된 단어 -> 학습
# 단어의 앞 3글자 입력 => 마지막 글자 예측
'''
wood => 학습 => 모델
woo 입력 =====> 모델 => d 예측
wop 입력 =====> 모델 => d 예측
'''

In [38]:
# chr함수 : 10진수로 ASCII코드 불러내기
char_arr = [chr(i) for i in range(97,123)]

num_dic = {}
num = 0
for i in char_arr :
    num_dic[i] = num
    num += 1
num_dic

num_dic={n:i for i, n in enumerate(char_arr)}
num_dic

dic_len = len(num_dic)

In [40]:
seq_data = ['word','wood','deep','dive','cold','cool','load','love','kiss','kind']

# < 목표 >
# x:wor, y = d
# x:woo, y = d

In [185]:
# seq 데이터를 전달받아 원핫인코딩 한 후 앞 세글자를 x로 뒤 한 글자를 y로 

def make_batch(seq_data) :
    
    input_batch = []
    target_batch = []
    
    for seq in seq_data :
        my_input =[num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        
        # 원핫인코딩
        input_batch.append(np.eye(dic_len)[my_input])
        target_batch.append(target)
        
    return input_batch, target_batch
    

In [188]:
input_batch, target_batch = make_batch(seq_data)

In [189]:
target_batch

[3, 3, 15, 4, 3, 11, 3, 4, 18, 3]

In [190]:
# softmax_corss_entropy_with_logist 함수는 label이 원핫 인코딩 돼있을 때
# sparse_softmax_corss_entropy_with_logist 함수는 label이 원핫 인코딩 안 됐을 때

In [265]:
# 변수
# lr=0.01, n_hidden=128, epoch=30, n_steps=3, n_input=n_class=26=dic_len
lr = 0.01
n_hidden = 128   # 출력 셀의 개수
total_epoch = 30
n_steps = 3      # 입력 3글자 (일반적으로 가장 긴 단어의 글자수를 주고 나머지는 zero padding)
n_input = n_class = dic_len  # 입력과 출력값의 dim

In [266]:
tf.reset_default_graph()
# 신경망 모델 구성                     단어수   입력글자수  26개의 종류
x = tf.placeholder(tf.float32, shape=[None, n_steps, n_input])
y = tf.placeholder(tf.int32, shape=[None])  # => [3], [5] .. 
# y가 원핫인코딩이 돼있다면        shape=[None, n_class] => [0,0,0,1,0,0...]

w = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, 0.5)  # Drop out 옵션 (오버피팅 방지)

cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, _states = tf.nn.dynamic_rnn(multi_cell, x, dtype=tf.float32)

In [267]:
outputs = tf.transpose(outputs, [1,0,2])
# 10,3,128 => 3,10,128
outputs = outputs[-1]  # 10, 128

model = tf.matmul(outputs, w) + b
# tf.matmul([10,128],[128,26]) => [10,26]

In [268]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=y))
opt = tf.train.AdamOptimizer(lr).minimize(cost)

In [269]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(total_epoch) :
    _, cv = sess.run([opt,cost], feed_dict={x:input_batch, y:target_batch})
    
    print('에폭 : %04d'%(epoch+1), '비용 : {:.5f}'.format(cv))
    
print('모델작성 완료')


에폭 : 0001 비용 : 2.92489
에폭 : 0002 비용 : 2.20015
에폭 : 0003 비용 : 2.01612
에폭 : 0004 비용 : 1.16369
에폭 : 0005 비용 : 1.11578
에폭 : 0006 비용 : 0.53239
에폭 : 0007 비용 : 1.37035
에폭 : 0008 비용 : 1.38564
에폭 : 0009 비용 : 1.27288
에폭 : 0010 비용 : 1.35574
에폭 : 0011 비용 : 0.98233
에폭 : 0012 비용 : 0.85629
에폭 : 0013 비용 : 0.51626
에폭 : 0014 비용 : 0.54353
에폭 : 0015 비용 : 0.70600
에폭 : 0016 비용 : 0.63348
에폭 : 0017 비용 : 0.64992
에폭 : 0018 비용 : 0.53844
에폭 : 0019 비용 : 1.10667
에폭 : 0020 비용 : 0.31352
에폭 : 0021 비용 : 0.33166
에폭 : 0022 비용 : 0.44037
에폭 : 0023 비용 : 0.67002
에폭 : 0024 비용 : 0.67361
에폭 : 0025 비용 : 0.31416
에폭 : 0026 비용 : 0.70809
에폭 : 0027 비용 : 0.53130
에폭 : 0028 비용 : 0.81074
에폭 : 0029 비용 : 0.50393
에폭 : 0030 비용 : 0.95269
모델작성 완료


In [270]:
# 예측 노드
pred = tf.cast(tf.argmax(model,1), tf.int32)

# 결과 비교
predCheck = tf.equal(pred, y)

# 정확도
accuracy = tf.reduce_mean(tf.cast(predCheck, tf.float32))

# input_batch, target_batch = make_batch(seq_data)

pv, av = sess.run([pred, accuracy], feed_dict={x:input_batch, y:target_batch})

In [271]:
predict_words = []
for i, v in enumerate(seq_data) :
    last_char = char_arr[pv[i]]
    predict_words.append(v[:3]+last_char)
print('예측결과\n')
print('입력값 : ',[w[:3] for w in seq_data])
print('예측값 : ',predict_words)
print('정확도 : ',av)

예측결과

입력값 :  ['wor', 'woo', 'dee', 'div', 'col', 'coo', 'loa', 'lov', 'kis', 'kin']
예측값 :  ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'loae', 'lovd', 'kiss', 'kind']
정확도 :  0.8


In [272]:
seq_test = ['deap','luve']
input_x, target_y = make_batch(seq_test)

In [273]:
pv, av = sess.run([pred, accuracy], feed_dict={x:input_x, y:target_y})

predict_words = []
for i, v in enumerate(seq_test) :
    last_char = char_arr[pv[i]]
    predict_words.append(v[:3]+last_char)
print('예측결과\n')
print('입력값 : ',[w[:3] for w in seq_test])
print('예측값 : ',predict_words)
print('정확도 : ',av)

예측결과

입력값 :  ['dea', 'luv']
예측값 :  ['deap', 'luve']
정확도 :  1.0


# seq2seq : 기계번역

 : 입력신경망(Encoder) / 출력신경망(Decoder)
ex) 나는 학교에 간다 => I go to school

: 챗봇, 번역, 이미지 캡셔닝에서 사용됨

In [312]:
'''
S : 디코딩 입력의 시작
E : 디코딩 출력의 끝
P : 현재 배치되는 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼

배치 데이터의 최대 크기 4인 경우
word = ['w','o','r','d']
to   = ['t','o','P','P']  # 길이 4에 맞춰 패딩됨
'''

# 학습에 사용할 단어 배열
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']  # 일반적으론 corpus

num_dic = {n:i for i,n in enumerate(char_arr)}
dic_len = len(num_dic)  # 41

seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'],
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data: # ['word', '단어']
        inputdata = [num_dic[n] for n in seq[0]]  # word
        outputdata = [num_dic[n] for n in ('S' + seq[1])]  # S단어
        targetdata = [num_dic[n] for n in (seq[1] + 'E')]  # 단어E

        input_batch.append(np.eye(dic_len)[inputdata])
        output_batch.append(np.eye(dic_len)[outputdata])
        target_batch.append(targetdata)

    return input_batch, output_batch, target_batch

input_batch, output_batch, target_batch = make_batch(seq_data)

In [329]:
tf.reset_default_graph()

# 모델 옵션설정
lr = 0.01
n_hidden = 128
total_epoch = 100
n_input = n_class = dic_len  # 41

In [330]:
# 신경망 구성 = [배치사이즈, 단계, 입력크기]
encInput = tf.placeholder(tf.float32, [None, None, n_input])   # word
decInput = tf.placeholder(tf.float32, [None, None, n_input])  # S단어
targets = tf.placeholder(tf.int64, [None, None])              # 단어E

In [332]:
# 인코더 셀을 구성
with tf.variable_scope('encode') :
    enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    # 위로       옆으로
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, encInput, dtype=tf.float32)


# 디코더 셀 구성
with tf.variable_scope('decode') :
    dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    #                                                                입력에서 넘어온 값
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, decInput, initial_state=enc_states, dtype=tf.float32)

In [333]:
model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

opt = tf.train.AdamOptimizer(lr).minimize(cost)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [334]:
# 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch) :
    _, cv = sess.run([opt, cost], feed_dict={encInput:input_batch ,decInput:output_batch, targets:target_batch})
    
    print('에폭 : %04d'%(epoch+1), '비용 : {:.5f}'.format(cv))


에폭 : 0001 비용 : 3.71407
에폭 : 0002 비용 : 3.59273
에폭 : 0003 비용 : 3.42726
에폭 : 0004 비용 : 3.02099
에폭 : 0005 비용 : 2.33814
에폭 : 0006 비용 : 2.35971
에폭 : 0007 비용 : 2.18335
에폭 : 0008 비용 : 1.71671
에폭 : 0009 비용 : 1.67572
에폭 : 0010 비용 : 1.82902
에폭 : 0011 비용 : 1.60616
에폭 : 0012 비용 : 1.30773
에폭 : 0013 비용 : 1.19909
에폭 : 0014 비용 : 0.94755
에폭 : 0015 비용 : 0.90957
에폭 : 0016 비용 : 0.92235
에폭 : 0017 비용 : 0.69990
에폭 : 0018 비용 : 0.76908
에폭 : 0019 비용 : 0.59338
에폭 : 0020 비용 : 0.76664
에폭 : 0021 비용 : 0.61363
에폭 : 0022 비용 : 0.47909
에폭 : 0023 비용 : 0.51138
에폭 : 0024 비용 : 0.41701
에폭 : 0025 비용 : 0.56903
에폭 : 0026 비용 : 0.32239
에폭 : 0027 비용 : 0.26971
에폭 : 0028 비용 : 0.28315
에폭 : 0029 비용 : 0.28808
에폭 : 0030 비용 : 0.19545
에폭 : 0031 비용 : 0.13966
에폭 : 0032 비용 : 0.13641
에폭 : 0033 비용 : 0.13492
에폭 : 0034 비용 : 0.14593
에폭 : 0035 비용 : 0.13029
에폭 : 0036 비용 : 0.12888
에폭 : 0037 비용 : 0.07538
에폭 : 0038 비용 : 0.11554
에폭 : 0039 비용 : 0.09771
에폭 : 0040 비용 : 0.05219
에폭 : 0041 비용 : 0.06621
에폭 : 0042 비용 : 0.03052
에폭 : 0043 비용 : 0.09894
에폭 : 0044 비

In [351]:
# 테스트 함수
def translate(w) :
    seq_data = [w, 'P'*len(w)]  # ['word', 'PPPP']
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    # input_batch = ['w','o','r','d']
    # output_batch = ['P','P','P','P']
    # target_batch = [2,2,2,2]
    
    #                       0      1      2
    # model실행 결과 : [배치사이즈, 스텝, 입력크기]
    prediction = tf.argmax(model, 2)  # 2번째 차원인 입력크키에 argmax 적용 => 확률이 가장 높은 글자 예측
    #                                        글자 인덱스
    # [[[0, 0, 0.9, 0.5, ,,, 0.1]]]  => [[[2],[3],,,[14]]]
    # ['단','어','E','E']
    
    res = sess.run(prediction, feed_dict={encInput:input_batch ,decInput:output_batch, targets:target_batch})
    
    decoded = [char_arr[i] for i in res[0]]
    # E 제거
    end = decoded.index('E')
    
    translated = ''.join(decoded[:end])
    
    return translated

In [363]:
print('word?', translate('word'))
print('wodr?', translate('wodr'))
print('love?', translate('love'))
print('loev?', translate('loev'))
print('translate?', translate('translate'))

word? 단어
wodr? 나무
love? 사랑
loev? 사랑
translate? 소소녀녀
